<a href="https://colab.research.google.com/github/L0ki2026/MarketHarvest/blob/main/MarketHarvest_trainingPipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Download Data

In [ ]:
# Download Data from the web
!wget -O cit_det.zip "https://mavmatrix.uta.edu/context/cse_datasets/article/1000/type/native/viewcontent"
!wget -O apple_data.tar.gz "https://conservancy.umn.edu/bitstreams/3ef26f04-6467-469b-9857-f443ffa1bb61/download"

# Unzip the data
!unzip /content/cit_det.zip
!tar -xvzf /content/apple_data.tar.gz
!unzip /content/CitDet-test.zip
!unzip /content/CitDet-train.zip

# Remove zip files
!rm /content/CitDet-test.zip /content/CitDet-train.zip /content/cit_det.zip /content/MANIFEST.TXT
!rm /content/apple_data.tar.gz

# Create file structure
!mkdir citrus_data
!mv /content/test citrus_data
!mv /content/train citrus_data
!mv detection apple_data
!mv /content/citrus_data/test/images/* /content/citrus_data/train/images/
!mv /content/citrus_data/test/masks/* /content/citrus_data/train/masks/

--2025-02-07 01:42:20--  https://mavmatrix.uta.edu/context/cse_datasets/article/1000/type/native/viewcontent
Resolving mavmatrix.uta.edu (mavmatrix.uta.edu)... 13.57.92.51, 50.18.241.247
Connecting to mavmatrix.uta.edu (mavmatrix.uta.edu)|13.57.92.51|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://mavmatrix.uta.edu/cgi/viewcontent.cgi?params=/context/cse_datasets/article/1000/type/native/&path_info= [following]
--2025-02-07 01:42:22--  https://mavmatrix.uta.edu/cgi/viewcontent.cgi?params=/context/cse_datasets/article/1000/type/native/&path_info=
Reusing existing connection to mavmatrix.uta.edu:443.
HTTP request sent, awaiting response... 200 OK
Length: 1103158596 (1.0G) [application/zip]
Saving to: ‘cit_det.zip’

cit_det.zip          59%[==========>         ] 621.28M  17.0MB/s    eta 28s    ^C
--2025-02-07 01:43:01--  https://conservancy.umn.edu/bitstreams/3ef26f04-6467-469b-9857-f443ffa1bb61/download
Resolving conservancy.umn.edu (con

## 1. Create Bounding Boxes

In [ ]:
import os
import torch
from torchvision.io import read_image
from torchvision.ops import masks_to_boxes
import numpy as np

# Paths
img_folder = r"/content/apple_data/train/images"
mask_folder = r"/content/apple_data/train/masks"
output_folder = r"/content/apple_data/train/masks"


# Iterate through all images and masks
for mask_file in os.listdir(mask_folder):
    # Ensure it's a valid image file
    if not mask_file.endswith(('.png', '.jpg', '.jpeg', '.tiff')):
        continue

    # Paths for the current image and mask
    mask_path = os.path.join(mask_folder, mask_file)
    img_path = os.path.join(img_folder, mask_file)

    # Read the mask
    mask = read_image(mask_path)

    # Get unique object IDs (skipping background)
    obj_ids = torch.unique(mask)
    obj_ids = obj_ids[1:]

    # Create binary masks for each object
    masks = mask == obj_ids[:, None, None]

    # Generate bounding boxes
    boxes = masks_to_boxes(masks)

    # Normalize boxes for YOLO format
    img_height, img_width = mask.shape[1:]  # H, W
    yolo_boxes = []
    for box in boxes:
        x_min, y_min, x_max, y_max = box
        x_center = (x_min + x_max) / 2 / img_width
        y_center = (y_min + y_max) / 2 / img_height
        width = (x_max - x_min) / img_width
        height = (y_max - y_min) / img_height
        yolo_boxes.append([0, x_center.item(), y_center.item(), width.item(), height.item()])

    # Save annotations to a YOLO format file
    yolo_file = os.path.join(output_folder, os.path.splitext(mask_file)[0] + ".txt")
    with open(yolo_file, "w") as f:
        for box in yolo_boxes:
            f.write(" ".join(map(str, box)) + "\n")

print(f"Annotations saved in YOLO format to: {output_folder}")


Annotations saved in YOLO format to: /content/apple_data/train/masks


In [ ]:
# for citrus
import cv2
from skimage.measure import label, regionprops

# Paths
mask_folder = r"/content/citrus_data/train/masks"
output_folder = r"/content/citrus_data/train/masks"

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Process each mask
for mask_file in os.listdir(mask_folder):
    if not mask_file.endswith(('.png', '.jpg', '.jpeg', '.tiff')):
        continue

    # Read the mask as a grayscale image
    mask_path = os.path.join(mask_folder, mask_file)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Apply connected component labeling
    labeled_mask = label(mask > 1)  # All greater than 1 pixels are considered part of a component

    # Extract bounding boxes from labeled regions
    img_height, img_width = mask.shape
    yolo_boxes = []

    for region in regionprops(labeled_mask):
        # Get bounding box (min_row, min_col, max_row, max_col)
        min_row, min_col, max_row, max_col = region.bbox

        # Convert to YOLO format (x_center, y_center, width, height)
        x_center = ((min_col + max_col) / 2) / img_width
        y_center = ((min_row + max_row) / 2) / img_height
        width = (max_col - min_col) / img_width
        height = (max_row - min_row) / img_height

        yolo_boxes.append([1, x_center, y_center, width, height])  # Class ID is set to 1

    # Save annotations in YOLO format
    yolo_file = os.path.join(output_folder, os.path.splitext(mask_file)[0] + ".txt")
    with open(yolo_file, "w") as f:
        for box in yolo_boxes:
            f.write(" ".join(map(str, box)) + "\n")

print(f"Annotations saved in YOLO format to: {output_folder}")


Annotations saved in YOLO format to: /content/citrus_data/train/masks


In [ ]:
# Remove segmentation masks
!rm -rf /content/citrus_data/train/masks/*.png
!rm -rf /content/apple_data/train/masks/*.png

# Create final file structure
!mkdir -p combined_data/images/train combined_data/images/val combined_data/labels/train combined_data/labels/val

# Move images and annotations into the correct folder
!mv /content/citrus_data/train/images/* /content/combined_data/images/train
!mv /content/apple_data/train/images/* /content/combined_data/images/train

!mv /content/apple_data/train/masks/* /content/combined_data/labels/train/
!mv /content/citrus_data/train/masks/* /content/combined_data/labels/train/

# Remove old files
!rm -rf /content/citrus_data /content/apple_data

In [ ]:
# Example YAML content
yaml_content = """
path: /content/combined_data
train: images/train
val: images/val


# Classes (80 COCO classes)
names:
    0: apple
    1: citrus
"""

# Write to a YAML file
with open("config.yaml", "w") as file:
    file.write(yaml_content)

print("YAML file created!")

YAML file created!


## 2. Split into training and testing set

In [ ]:
import random
import shutil

# Set your dataset paths
images_path = r"/content/combined_data/images/train"
annotations_path = r"/content/combined_data/labels/train"

# Destination paths
train_images_dest = r"/content/combined_data/images/train"
val_images_dest = r"/content/combined_data/images/val"
train_annotations_dest = r"/content/combined_data/labels/train"
val_annotations_dest = r"/content/combined_data/labels/val"

# Create necessary folders
os.makedirs(train_images_dest, exist_ok=True)
os.makedirs(val_images_dest, exist_ok=True)
os.makedirs(train_annotations_dest, exist_ok=True)
os.makedirs(val_annotations_dest, exist_ok=True)

# Get the list of all images and shuffle them
images = [f for f in os.listdir(images_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
random.shuffle(images)

# Split the data into 80% train and 20% val
split_ratio = 0.80
split_index = int(len(images) * split_ratio)

train_images = images[:split_index]
val_images = images[split_index:]

# Function to move files
def move_files(file_list, src_images_path, src_annotations_path, dest_images_path, dest_annotations_path):
    for image in file_list:
        # Move image
        shutil.move(os.path.join(src_images_path, image), os.path.join(dest_images_path, image))

        # Move corresponding annotation
        annotation_file = image.replace('.png', '.txt').replace('.jpg', '.txt').replace('.jpeg', '.txt')
        shutil.move(os.path.join(src_annotations_path, annotation_file), os.path.join(dest_annotations_path, annotation_file))

# Move train files
move_files(train_images, images_path, annotations_path, train_images_dest, train_annotations_dest)

# Move val files
move_files(val_images, images_path, annotations_path, val_images_dest, val_annotations_dest)

print("Train-Test split completed successfully!")


Train-Test split completed successfully!


## 3. Train Model

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 910.2/910.2 kB 29.3 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
import torch

model = YOLO("yolo11m.pt")

if __name__ == "__main__":

  if torch.cuda.is_available():
    results = model.train(data="config.yaml", epochs=100, imgsz=320, batch=4, device=0, patience=10)
  else:
    results = model.train(data="config.yaml", epochs=100, imgsz=320, batch=4, patience=10)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 38.8M/38.8M [00:00<00:00, 233MB/s]


Ultralytics 8.3.63 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=config.yaml, epochs=100, time=None, patience=10, batch=4, imgsz=320, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=N

100%|██████████| 755k/755k [00:00<00:00, 20.3MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  7                  -1  1   2360320  ultralytics

...

                   all        250      13172       0.65      0.452      0.502      0.222

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     28/100      1.15G      2.142      1.072     0.8189        249        320: 100%|██████████| 250/250 [00:57<00:00,  4.35it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:08<00:00,  3.88it/s]


                   all        250      13172      0.633      0.452      0.495      0.214
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 18, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

28 epochs completed in 0.542 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 40.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 40.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.63 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11m summary (fused): 303 layers, 20,031,574 parameters, 0 gradients, 67.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:12<00:00,  2.61it/s]


                   all        250      13172      0.664      0.457      0.505      0.224
                 apple        139       5690      0.701      0.583      0.631      0.269
                citrus        111       7482      0.626       0.33       0.38      0.179
Speed: 0.2ms preprocess, 6.9ms inference, 0.0ms loss, 3.1ms postprocess per image
Results saved to runs/detect/train


In [ ]:
model_path = r"/content/model"
os.makedirs(model_path, exist_ok=True)

In [ ]:
model.save(model_path+"/trained_yolo_model.pt")